In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm

In [46]:
# Upload all datasets
word_categorization = pd.read_csv("word_categorization.csv")
word_recognition = pd.read_csv("word_recognition.csv")
word_frequency = pd.read_csv("word_frequency_tot_categorization_task.csv")
wc_mt_statistics = pd.read_csv("WC_MT_statistics.csv", header=[0])

In [10]:
df_user_10106 = word_categorization[word_categorization['user_id'] == 10106]
#df_user_10106

# VCheck for non-null values in column ‘response_time_ms’ for user 10106
non_nan_response_times = df_user_10106['response_time_ms'].dropna()

# Count how many non-zero values there are
num_non_nan_values = non_nan_response_times.count()

num_non_nan_values

0

In [36]:
word_categorization.head()

,user_id,voucher,valid,language,word,user_given_response,expected_correct_response,is_user_response_correct,alive_notAlive,manmade_notManmade,response_time_ms
0,10106,1,1,0,HIGHWAY,0,0,1,0.0,NaN,NaN
1,10106,1,1,0,CLOVER,0,1,0,1.0,NaN,NaN
2,10106,1,1,0,KING,1,1,1,1.0,NaN,NaN
3,10106,1,1,0,NAPKIN,0,0,1,0.0,NaN,NaN
4,10106,1,1,0,BARREL,0,0,1,0.0,NaN,NaN


In [47]:
wc_mt_statistics.head() # 51 to go beyond null

,user_id,voucher,valid,language,WC_CorrAns_RT_mean_1,WC_CorrAns_RT_SD_1,WC_correct_answers_percentage_1,WC_information_1,WC_information_rate_1,MT_IC_1,MT_EC_1,MT_IW_1,MT_EW_1,MT_NG_1,MT_C_1
0,10106,1,1,0,NaN,NaN,83.076923,0.344058,NaN,14.0,31.5,11.5,19.0,24.0,45.5
1,10162,1,1,0,NaN,NaN,93.846154,0.666478,NaN,55.0,1.0,25.5,2.5,16.0,56.0
2,100,1,1,0,NaN,NaN,96.923077,0.801765,NaN,14.5,65.0,10.0,4.0,6.5,79.5
3,10154,1,1,0,NaN,NaN,92.307692,0.608756,NaN,0.0,64.0,0.5,34.0,1.5,64.0
4,10144,1,1,0,NaN,NaN,94.615385,0.697484,NaN,19.0,40.5,11.0,6.5,23.0,59.5


In [59]:
# Check for the presence of NaN
nan_summary = wc_mt_statistics.isnull().sum()
nan_columns = nan_summary[nan_summary > 0]

# Show columns with NaN values and their amount
print("Columns with NaN values and their amount:")
print(nan_columns)

# Options for dealing with NaN values
print("\nOptions for dealing with NaN values:")
print("1. Delete rows with NaN: df.dropna(inplace=True)")
print("2. Fill with the average of the column: df.fillna(df.mean(), inplace=True)")
print("3. Fill with interpolation: df.interpolate(inplace=True)")
print("4. Fill with a specific value, e.g. 0: df.fillna(0, inplace=True)")

Columns with NaN values and their amount:
WC_CorrAns_RT_mean_1     49
WC_CorrAns_RT_SD_1       49
WC_information_rate_1    49
dtype: int64

Options for dealing with NaN values:
1. Delete rows with NaN: df.dropna(inplace=True)
2. Fill with the average of the column: df.fillna(df.mean(), inplace=True)
3. Fill with interpolation: df.interpolate(inplace=True)
4. Fill with a specific value, e.g. 0: df.fillna(0, inplace=True)


In [ ]:
# hacer la mediana de los valores que faltan
# doing the median for the null values (inplace =True)
# train_data.Mri_Track_age_scan.fillna(train_data.Mri_track_age_scan.median(), inplace=True)

In [15]:
# Count the number of unique user_id's in the file
unique_user_ids = word_categorization['user_id'].nunique()
unique_user_ids

235

### VERIFICATION OF REACTION TIMES

In [30]:
# Redondear a 3 decimales los valores antes de la comparación
df_comparison_filtered["WC_CorrAns_RT_mean_1"] = df_comparison_filtered["WC_CorrAns_RT_mean_1"].round(3)
df_comparison_filtered["calculated_WC_CorrAns_RT_mean_1"] = df_comparison_filtered["calculated_WC_CorrAns_RT_mean_1"].round(3)

df_comparison_filtered["WC_CorrAns_RT_SD_1"] = df_comparison_filtered["WC_CorrAns_RT_SD_1"].round(3)
df_comparison_filtered["calculated_WC_CorrAns_RT_SD_1"] = df_comparison_filtered["calculated_WC_CorrAns_RT_SD_1"].round(3)

df_comparison_filtered["WC_correct_answers_percentage_1"] = df_comparison_filtered["WC_correct_answers_percentage_1"].round(3)
df_comparison_filtered["calculated_WC_correct_answers_percentage_1"] = df_comparison_filtered["calculated_WC_correct_answers_percentage_1"].round(3)

# Compare if values match to 3 decimal places
df_comparison_filtered["mean_match"] = df_comparison_filtered["WC_CorrAns_RT_mean_1"] == df_comparison_filtered["calculated_WC_CorrAns_RT_mean_1"]
df_comparison_filtered["std_match"] = df_comparison_filtered["WC_CorrAns_RT_SD_1"] == df_comparison_filtered["calculated_WC_CorrAns_RT_SD_1"]
df_comparison_filtered["accuracy_match"] = df_comparison_filtered["WC_correct_answers_percentage_1"] == df_comparison_filtered["calculated_WC_correct_answers_percentage_1"]

# Coincidences and disagreements
mean_match_count_filtered = df_comparison_filtered["mean_match"].sum()
std_match_count_filtered = df_comparison_filtered["std_match"].sum()
accuracy_match_count_filtered = df_comparison_filtered["accuracy_match"].sum()

total_users_filtered = df_comparison_filtered.shape[0]

{
    "Total users (without NaN values)": total_users_filtered,
    "Mean reaction time matches (3 decimals)": mean_match_count_filtered,
    "STD reaction time matches (3 decimals)": std_match_count_filtered,
    "Accuracy matches (3 decimals)": accuracy_match_count_filtered
}


{'Total users (without NaN values)': 186,
 'Mean reaction time matches (3 decimals)': 186,
 'STD reaction time matches (3 decimals)': 0,
 'Accuracy matches (3 decimals)': 186}

In [58]:
# Recalculate the standard deviation with ddof=0 to check if it is a better match
df_correct_responses = word_categorization[word_categorization["is_user_response_correct"] == 1]

# Calculate the standard deviation with ddof=0
reaction_time_stats_ddof0 = df_correct_responses.groupby("user_id")["response_time_ms"].agg(
    mean="mean", std=lambda x: x.std(ddof=0)
).reset_index()

# Rename columns for comparison
reaction_time_stats_ddof0.rename(columns={'mean': 'calculated_WC_CorrAns_RT_mean_1', 'std': 'calculated_WC_CorrAns_RT_SD_1'}, inplace=True)

# Merge with the filtered table WC_MT_statistics to compare
df_comparison_filtered_ddof0 = df_filtered.merge(reaction_time_stats_ddof0, on="user_id", how="left")

# Round to 3 decimal places before comparing
df_comparison_filtered_ddof0["WC_CorrAns_RT_SD_1"] = df_comparison_filtered_ddof0["WC_CorrAns_RT_SD_1"].round(3)
df_comparison_filtered_ddof0["calculated_WC_CorrAns_RT_SD_1"] = df_comparison_filtered_ddof0["calculated_WC_CorrAns_RT_SD_1"].round(3)

# Compare if values match to 3 decimal places
df_comparison_filtered_ddof0["std_match"] = df_comparison_filtered_ddof0["WC_CorrAns_RT_SD_1"] == df_comparison_filtered_ddof0["calculated_WC_CorrAns_RT_SD_1"]

# Counting coincidences and discrepancies in standard deviation
std_match_count_filtered_ddof0 = df_comparison_filtered_ddof0["std_match"].sum()

# Verify the result
std_match_count_filtered_ddof0, df_comparison_filtered_ddof0.shape[0]


(186, 186)

In [32]:
# NOTE: it is compared with 3 decimals because few results matched with the default ones but using 3 decimals
# all the registers coincide except for the NaN which are not calculated.

In [ ]:
# NOTE: The problem was due to the fact that the original calculation in the database used the population standard deviation (ddof=0), while the standard method in Pandas (ddof=1) calculates the sample standard deviation. 
#(ddof=0), while the standard method in Pandas (ddof=1) calculates the sample standard deviation, 
#, which generated small differences.

In [49]:
# Ensure 'user_id' is of integer type in both DataFrames
wc_mt_statistics["user_id"] = wc_mt_statistics["user_id"].astype(int)
word_categorization["user_id"] = word_categorization["user_id"].astype(int)

# Retry the calculation for user 10150
user_10150_stored_value = wc_mt_statistics[wc_mt_statistics["user_id"] == 10150]["WC_CorrAns_RT_SD_1"].values

# Get the reaction times for correct responses for user 10150
user_10150_reaction_times = word_categorization[
    (word_categorization["user_id"] == 10150) & 
    (word_categorization["is_user_response_correct"] == 1)
]["response_time_ms"]

# Compute the standard deviation
user_10150_calculated_sd = user_10150_reaction_times.std()

# Display results
{
    "Stored WC_CorrAns_RT_SD_1": user_10150_stored_value[0] if len(user_10150_stored_value) > 0 else None,
    "Calculated WC_CorrAns_RT_SD_1": user_10150_calculated_sd
}


{'Stored WC_CorrAns_RT_SD_1': 761.1226641432567,
 'Calculated WC_CorrAns_RT_SD_1': 764.2103955403774}

In [54]:
user_10150_calculated_sd_sample = user_10150_reaction_times.std(ddof=1)  # Sample (n-1)
user_10150_calculated_sd_population = user_10150_reaction_times.std(ddof=0)  # Population (n)
user_10150_calculated_sd_sample, user_10150_calculated_sd_population

(764.2103955403774, 761.1226641432567)